In [157]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from scipy.optimize import differential_evolution

In [158]:
org_df = pd.read_csv('waterDataTraining.csv')
df = org_df.copy()
# df.dropna(inplace = True)
df.head()

,Time,Tp,pH,Cond,Turb,SAC,PFM,Event
0,2017-07-01 00:00:00,6.94,8.60774,0.020954,0.125931,3.58683,43.7559,False
1,2017-07-01 00:01:00,6.93,8.60589,0.020965,0.127219,3.59025,43.4366,False
2,2017-07-01 00:02:00,6.94,8.60220,0.020968,0.126482,3.58318,43.5994,False
3,2017-07-01 00:03:00,6.94,8.60220,0.020972,0.126184,3.58769,43.3704,False
4,2017-07-01 00:04:00,6.94,8.60405,0.020973,0.127908,3.58287,43.1656,False


In [159]:
df['Event'].value_counts()

False    132268
True        212
Name: Event, dtype: int64

In [160]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 132480 entries, 0 to 132479
Data columns (total 8 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   Time    132480 non-null  object 
 1   Tp      132266 non-null  float64
 2   pH      132272 non-null  float64
 3   Cond    132276 non-null  float64
 4   Turb    132275 non-null  float64
 5   SAC     132280 non-null  float64
 6   PFM     132269 non-null  float64
 7   Event   132480 non-null  bool   
dtypes: bool(1), float64(6), object(1)
memory usage: 7.2+ MB


In [161]:
# df['Tp_m_avg'] 	= df['Tp'].rolling(window = 30).mean()
# df['pH_m_avg'] 	= df['pH'].rolling(window = 30).mean()
# df['Turb_m_avg'] = df['Turb'].rolling(window = 30).mean()
# df['SAC_m_avg'] 	= df['SAC'].rolling(window = 30).mean()
# df['PFM_m_avg'] 	= df['PFM'].rolling(window = 30).mean()
# df['Cond_m_avg'] = df['Cond'].rolling(window = 30).mean()

In [162]:
# df.dropna(inplace = True)
# df = org_df.copy()
# df2 = df.copy()
# df2['Tp'] = df['Tp'].fillna(df['Tp'].median())
# df2['pH'] = df['pH'].fillna(df['pH'].median())
# df2['Cond'] = df['Cond'].fillna(df['Cond'].median())
# df = df.fillna(df.median(numeric_only = True))
df['Tp_m_avg'] 	= df['Tp'].rolling(window = 30).median()
df['pH_m_avg'] 	= df['pH'].rolling(window = 30).median()
df['Turb_m_avg'] = df['Turb'].rolling(window = 30).median()
df['SAC_m_avg'] 	= df['SAC'].rolling(window = 30).median()
df['PFM_m_avg'] 	= df['PFM'].rolling(window = 30).median()
df['Cond_m_avg'] = df['Cond'].rolling(window = 30).median()


df['Tp_m_min'] 	= df['Tp'].rolling(window = 30).min()
df['pH_m_min'] 	= df['pH'].rolling(window = 30).min()
df['Turb_m_min'] = df['Turb'].rolling(window = 30).min()
df['SAC_m_min'] 	= df['SAC'].rolling(window = 30).min()
df['PFM_m_min'] 	= df['PFM'].rolling(window = 30).min()
df['Cond_m_min'] = df['Cond'].rolling(window = 30).min()


df['Tp_m_mam'] 	= df['Tp'].rolling(window = 30).max()
df['pH_m_mam'] 	= df['pH'].rolling(window = 30).max()
df['Turb_m_mam'] = df['Turb'].rolling(window = 30).max()
df['SAC_m_mam'] 	= df['SAC'].rolling(window = 30).max()
df['PFM_m_mam'] 	= df['PFM'].rolling(window = 30).max()
df['Cond_m_mam'] = df['Cond'].rolling(window = 30).max()
df.dropna(inplace = True)

In [163]:
X = df.drop(['Time', 'Event'], axis = 1)
y = df['Event']

In [164]:
from imblearn.over_sampling import SMOTE

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

print(" X_train dataset: ", X_train.shape)
print(" y_train dataset: ", y_train.shape)
print(" X_test dataset: ", X_test.shape)
print(" y_test dataset: ", y_test.shape)

 X_train dataset:  (87614, 24)
 y_train dataset:  (87614,)
 X_test dataset:  (37549, 24)
 y_test dataset:  (37549,)


In [165]:
sm = SMOTE(random_state=2)
X_train_res, y_train_res = sm.fit_resample(X_train, y_train.ravel())

In [183]:
from sklearn.metrics import f1_score, confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier

def get_model(x, model_choice):
# Build a Logistic Regression Model, fit the model to data and return the model.
# Hyper Parameter values are sent as parameter to the function.

    if model_choice == 0: 
        model = LogisticRegression( C = x[0], max_iter=1000, random_state = 123)
    elif:
        model = RandomForestClassifier(n_estimators = int(x[0]), random_state = 123)
    else:
        model = MLPClassifier(activation='tanh', solver='adam', alpha=x[0], 
                          hidden_layer_sizes=(int(x[1]), int(x[2])), random_state=1)
        
    model.fit(X_train, y_train)
    return get_model_f1(model)

def get_random_forest_model(x):
    model = RandomForestClassifier(n_estimators = int(x[0]), random_state = 123)
    model.fit(X_train_res, y_train_res)
    return get_model_f1(model)

def get_mlp_model(x):
    model = MLPClassifier(activation='tanh', solver='adam', alpha=x[0], 
                          hidden_layer_sizes=(int(x[1]), int(x[2])), random_state=1)
    model.fit(X_train_res, y_train_res)
    return get_model_f1(model)

def get_model_f1(model):
    # DE tries to minimize the objective function. Hence, I'm returning 1-f1 instead of f1 so that it minimises 1-f1 
    # which is the same as maximising f1.
    y_pred = model.predict(X_test)
    tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
    recall = tp / (tp + fn)
    precision = tp / (tp + fp)
    f1 = 2 * (precision * recall) / (precision + recall)
    return 1-f1
    

In [193]:
 def get_model(x, model_choice):
    if model_choice == 0: 
        model = LogisticRegression( C = x[0], max_iter=1000)
    elif model_choice == 1:
        model = RandomForestClassifier(n_estimators = int(x[0]))
    else:
        model = MLPClassifier(activation='tanh', solver='adam', alpha=x[0], 
                              hidden_layer_sizes=(int(x[1]), int(x[2])))
    model.fit(X_train, y_train)
    return get_model_f1(model)

def get_model_f1(model):
    # DE tries to minimize the objective function. Hence, I'm returning 1-f1 instead of f1 so that it minimises 1-f1 
    # which is the same as maximising f1.
    y_pred = model.predict(X_test)
    tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
    recall = tp / (tp + fn)
    precision = tp / (tp + fp)
    f1 = 2 * (precision * recall) / (precision + recall)
    return 1-f1

In [194]:
LR_bounds = [(100, 1000)] # Bounds for Logistic Regression
MLP_bounds = [(0, 1e-1), (1, 100), (1, 10)] # Bounds for MLP Classifier
RF_bounds = [(0, 1)] # Bounds for Random Forest Classifier
result = differential_evolution(func = get_logistic_model, bounds = bounds, maxiter = 10, popsize = 10)

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (2,) + inhomogeneous part.

In [184]:
LR_bounds = [(100, 1000)] # Bounds for Logistic Regression
MLP_bounds = [(0, 1e-1), (1, 100), (1, 10)] # Bounds for MLP Classifier
RF_bounds = [(0, 1)] # Bounds for Random Forest Classifier
# bounds = [(100, 1000)]
result = differential_evolution(func = get_logistic_model, bounds = bounds, maxiter = 10, popsize = 10)
print(result.x, result.fun)

C:\Users\yasas\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\yasas\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_

C:\Users\yasas\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\yasas\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_

[923.58560018] 0.40594059405940597


C:\Users\yasas\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [175]:
print(result.x, 1-result.fun)

[956.18019606] 0.9420289855072465


In [178]:
model = RandomForestClassifier(n_estimators = 956, random_state = 123)
model.fit(X_train_res, y_train_res)
y_pred = model.predict(X_test)
get_metrics(y_pred, y_test)

(0.999786945058457, 0.9285714285714286, 0.9558823529411765, 0.9420289855072465)

In [177]:
from sklearn.metrics import confusion_matrix
def get_metrics(Y_true, y_pred):
    tn, fp, fn, tp = confusion_matrix(Y_true, y_pred).ravel()
    acc = (tp + tn) / (tp + fp + tn + fn)
    recall = tp / (tp + fn)
    precision = tp / (tp + fp)
    f1 = 2 * (precision * recall) / (precision + recall)
    return acc, precision, recall, f1